# 패키지 불러오기

In [3]:
import sklearn
import pandas as pd
import numpy as np

In [ ]:
pip install surprise

# 저장된 meetings와 moim 데이터 불러오기와서 합침

In [12]:
meetings = pd.read_csv('./meeting_data.csv')
moim = pd.read_csv('./moim_data.csv')
df = pd.concat([meetings, moim])
df

,meeting_id,uid,writer,created_at,updated_at,is_period,meeting_date,period_date,is_class,max_person,...,ref_url,address,fee,unit,is_active,like_cnt,view_cnt,score,main_category,tags
0,108923,https://www.frip.co.kr/hosts/617,그 아저씨,2020-05-24 23:52:30.200913,2020-05-24 23:52:30.200913,0,2020-05-24 23:52:30.200913,NaN,1,20,...,https://www.frip.co.kr/products/108923,서울시 성동구 무학봉 15길 27 지하1층(지하철 2호선 상왕십리역 4번 출구에서 ...,32000,원,1,11152,0,4.77,0,NaN
1,133413,https://www.frip.co.kr/hosts/5545,MCJ,2020-05-24 23:52:35.026133,2020-05-24 23:52:35.026133,0,2020-05-26 19:30:00.000000,NaN,1,20,...,https://www.frip.co.kr/products/133413,서울시 강남구 학동로30길 45 힐하우스 3층,7500,원,1,221,0,5.00,0,NaN
2,101774,https://www.frip.co.kr/hosts/746,아이홉 맥주공방 (I'HOP),2020-05-24 23:52:39.366188,2020-05-24 23:52:39.366188,0,2020-05-31 13:00:00.000000,NaN,1,20,...,https://www.frip.co.kr/products/101774,서울시 송파구 백제고분로 243 삼전빌딩 지하 아이홉(I'HOP) 맥주공방(송파구의...,40000,원,1,9813,0,4.76,0,NaN
3,133023,https://www.frip.co.kr/hosts/10654,비푸머스,2020-05-24 23:52:43.862187,2020-05-24 23:52:43.862187,0,2020-05-24 23:52:43.862187,NaN,1,20,...,https://www.frip.co.kr/products/133023,"서울시 마포구 와우산로33길 23, 비푸머스",42750,원,1,805,0,4.79,0,NaN
4,115497,https://www.frip.co.kr/hosts/2839,지아네요리공작소,2020-05-24 23:52:48.661984,2020-05-24 23:52:48.661984,0,2020-05-24 23:52:48.661984,NaN,1,20,...,https://www.frip.co.kr/products/115497,"서울시 마포구 방울내로 80-16, 1층 지아네요리공작소",30000,원,1,5645,0,4.89,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,135609,https://www.frip.co.kr/hosts/11305,박초롱,2020-05-24 23:59:56.637987,2020-05-24 23:59:56.637987,0,2020-06-09 19:30:00.000000,NaN,1,20,...,https://www.frip.co.kr/products/135609,서울 마포구 동교로24길 17 3층 이코노미스타,149000,원,1,53,0,0.00,0,NaN
176,134752,https://www.frip.co.kr/hosts/11160,아페리,2020-05-24 23:59:59.748122,2020-05-24 23:59:59.748122,0,2020-06-06 19:00:00.000000,NaN,1,20,...,https://www.frip.co.kr/products/134752,"서울시 중구 을지로2가 199-46, 층일빌딩 3층(일마지오)",38000,원,1,97,0,0.00,0,NaN
177,133251,https://www.frip.co.kr/hosts/10766,수오재기,2020-05-25 00:00:03.899047,2020-05-25 00:00:03.899047,0,2020-06-01 00:00:00.000000,NaN,1,20,...,https://www.frip.co.kr/products/133251,"생활은 일상에서, 칭찬은 카카오톡에서 진행",10000,원,1,57,0,4.90,0,NaN
178,128055,https://www.frip.co.kr/hosts/5493,에스프레소,2020-05-25 00:00:07.198502,2020-05-25 00:00:07.198502,0,2020-05-29 19:45:00.000000,NaN,1,20,...,https://www.frip.co.kr/products/128055,"서울시 강남구 선릉로64길 28, 메리제인 플라워카페",20000,원,1,407,0,4.71,0,NaN


In [13]:
meeting_images = pd.read_csv('./meeting_images.csv')
moim_images = pd.read_csv('./moim_images.csv')
images = pd.concat([meeting_images, moim_images])
images

,Unnamed: 0,meeting_id,image_url
0,0,108923,https://res.cloudinary.com/frientrip/image/upl...
1,1,133413,None
2,2,101774,None
3,3,133023,None
4,4,115497,None
...,...,...,...
175,175,135609,None
176,176,134752,None
177,177,133251,None
178,178,128055,https://res.cloudinary.com/frientrip/image/upl...


# 합친 모임/강좌 정보와 이미지 링크 정보를 csv 파일로 저장

In [14]:
df.to_csv('meetings.csv', index=False, encoding='utf-8-sig')
images.to_csv('images.csv', index=False, encoding='utf-8-sig')